# Цветовые схемы. Преобразования цветовых схем.

In [ ]:
#import matplotlib
#matplotlib.use('qt5agg')
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np

import random
import pathlib

from matplotlib.colors import LinearSegmentedColormap

 *Преобразования в разные цветовые схемы: RGB, CMYK, HSV, CIE YCbCr*

\begin{equation}
e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i
\end{equation}

_**Inline formula:**_ $e^{i\pi} + 1 = 0$

In [ ]:
path = "img/medium"
collection = images = [p for p in pathlib.Path(path).iterdir() if p.suffix in [".jpg", ".jpeg"]]
img = plt.imread(random.choice(collection))

#figure, axes = plt.subplots()
plt.imshow(img, cmap='hsv')
plt.axis('off')
plt.show()

In [ ]:
colormaps = [
    ["Reds", "Greens", "Blues"],
    [
        LinearSegmentedColormap.from_list("BkR", ["Black", "Red"]),
        LinearSegmentedColormap.from_list("BkG", ["Black", "Green"]),
        LinearSegmentedColormap.from_list("BkBu", ["Black", "Blue"])
    ],
]

for cm in range(len(colormaps)):
    for ch in range(len(colormaps[cm])):
        plt.subplot(2, 3, cm*3 + ch+1).imshow(img[:, :, ch], cmap=colormaps[cm][ch])

for axes in plt.gcf().findobj(plt.Axes):
    axes.set_axis_off()
plt.show()

In [ ]:
# RGB cube
granularity = 8
edge_len= int(256 / granularity)
color_sl = complex(0, edge_len + 1)
space = np.mgrid[0:1:color_sl, 0:1:color_sl, 0:1:color_sl]
cube = (space[0] < 1) & (space[1] < 1) & (space[2] < 1)

colors = np.transpose(space, (1, 2, 3, 0))

plt.close(fig=1)
plt.figure().add_subplot(projection='3d').voxels(cube, facecolors=colors)
plt.show()

In [ ]:
#CMY cube

plt.close(fig=2)
plt.figure().add_subplot(projection='3d').voxels(cube, facecolors=(1 - colors), linewidth=0.5)
plt.show()

In [ ]:
import scipy.spatial.transform.rotation
#cube_rotation = scipy.spatial.transform.Rotation.from_euler("XY", (35, -45), degrees=True)
#cube_rotation = scipy.spatial.transform.Rotation.from_euler("xy", (45, -35), degrees=True)
z_axe_rot = -np.pi/6
cube_rotation = scipy.spatial.transform.Rotation.from_euler("xyz", (np.pi / 4, -np.arctan(1/np.sqrt(2)), z_axe_rot))
grid = np.transpose(space / np.sqrt(3), (1, 2, 3, 0))
hsv_space = np.reshape(cube_rotation.apply(np.reshape(grid, newshape=(-1, 3))), grid.shape)

plt.close(fig=3)
plt.figure(num=3).add_subplot(projection='3d')
plt.gca().voxels(hsv_space[..., 0], hsv_space[..., 1], hsv_space[..., 2], cube[:-1, :-1, :-1], facecolors=colors[1:, 1:, 1:], linewidth=0.5)
plt.gca().scatter([0], [0], [1.0], marker='o')
plt.show()

In [ ]:
inverse_transformation = np.reshape(cube_rotation.apply(np.reshape(hsv_space, newshape=(-1, 3)), inverse=True), grid.shape)
plt.figure().add_subplot(projection='3d')\
    .voxels(inverse_transformation[..., 0], inverse_transformation[..., 1], inverse_transformation[..., 2], cube, facecolors=colors)
plt.show()


In [ ]:
cylinder_shape = (
    complex(0, int(180 / granularity)),
    complex(0, int((180 / granularity) * 2 * np.pi)),
    complex(0, int(edge_len))
)
cylinder_size = (1.0, 2*np.pi, 1.0)
cylinder_grid = np.mgrid[0:cylinder_size[0]:cylinder_shape[0], 0:cylinder_size[1]:cylinder_shape[1], 0:cylinder_size[2]:cylinder_shape[2]]
corners = np.array(hsv_space[0:2, 0:2, 0:2, :] * (edge_len)).reshape((-1, 3)).astype(np.float64)
order = [0, 1, 3, 7, 6, 4, 0, 1]
planes = np.zeros(shape=(6, 4), dtype=np.float64)

for p in range(len(order) - 2):
    #print(f"{p+1}.\t{order[p]}. {corners[order[p]]}\n\t{order[p+1]}. {corners[order[p+1]]}\n\t{order[p+2]}. {corners[order[p+2]]}")
    planes[p][0:3] = np.cross(corners[order[p]] - corners[order[p+1]], corners[order[p+2]] - corners[order[p+1]])
    planes[p][3] = np.sum(planes[p][0:3] * corners[order[p+1]])

equations = np.zeros(shape=(6, 3, 4), dtype=np.float64)
equations[:, 0, :] = planes
#cylinder = ((cylinder_grid[0] * np.cos(cylinder_grid[1])) ** 2 + (cylinder_grid[0] * np.sin(cylinder_grid[1])) ** 2 < 0.25)
cylinder = cylinder_grid[0] < cylinder_size[0]
cylinder_grid = cylinder_grid.transpose((1, 2, 3, 0)).astype(np.float64)
cylinder_colors = np.zeros(shape=cylinder_grid.shape)

border_crossings = []
planed_vectors = []
crossings_dist = []
contained_points = []

__fill_voxels = np.full(cylinder.shape, False, dtype=bool)
for h in range(int(cylinder_shape[2].imag)-1): # decrease range by 1
    center = np.array([0, 0, cylinder_grid[0][0][h][2]])
    print(f"{center}")
    equations[:, 1, :] =  np.array([0, 0, 1, cylinder_grid[0][0][h][2]])
    for t in range(int(cylinder_shape[1].imag)-1): # decrease range by 1
        equations[:, 2, :] = np.array([np.cos(cylinder_grid[0][t][h][1] - np.pi/2 - z_axe_rot), -np.sin(cylinder_grid[0][t][h][1] - np.pi/2 - z_axe_rot), 0, 0])
        scale_points = np.linalg.solve(equations[:, :, 0:3], np.squeeze(equations[:, :, 3]))
        scale_norms = np.linalg.norm(scale_points - center, axis=1)
        direction_vector = np.array([np.cos(cylinder_grid[0][t][h][1] - z_axe_rot), -np.sin(cylinder_grid[0][t][h][1] - z_axe_rot), 0])
        for point in scale_points:
            crossings_dist.append(np.linalg.norm(point - center))
        border_crossings.append(scale_points[np.argmin(scale_norms)])
        border_crossings.append(scale_points[np.argmin(scale_norms)] * np.array([-1, -1, 1]))
        direction_mask = np.argwhere(np.all(scale_points * direction_vector >= 0, axis=1))
        d = np.amin(scale_norms[direction_mask])
        #print(f"{scale_norms}: min={d}; scale_points.shape={scale_points.shape}")
        rate = d / cylinder_size[0]
        for r in range(int(cylinder_shape[0].imag)): # decrease range by 1
            transformed_coordinates = np.array([
                cylinder_grid[r][t][h][0]*np.cos(cylinder_grid[r][t][h][1] + z_axe_rot)*rate,
                cylinder_grid[r][t][h][0]*np.sin(cylinder_grid[r][t][h][1] + z_axe_rot)*rate,
                cylinder_grid[r][t][h][2]
            ])
            contained_points.append(transformed_coordinates)
            index = cube_rotation.apply(transformed_coordinates*np.sqrt(3), inverse=True)
            index *=  edge_len
            index = index.astype(int)

            if np.all(index == np.array([31, 0, 0])):
                print(f"Red: ({r}, {t}, {h})")
            cylinder_colors[r][t][h] = colors[index[0]][index[1]][index[2]]

__boundary_points = np.array(border_crossings)
__contained_points = np.array(contained_points)
__corners = np.reshape(corners, newshape=(2, 2, 2, 3))

plt.close(fig=4)
plt.figure(num=4).add_subplot(projection='3d')
#plt.gca().voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), cylinder_grid[..., 2], cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1])
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__contained_points[0:-1:3,..., 0], __contained_points[0:-1:3,..., 1], __contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
plt.close(fig=5)
plt.figure(num=5).add_subplot(projection='3d')
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.show()

In [ ]:
__cp_indicies = np.argwhere(np.logical_and(__contained_points[:, 2] > 0.48, __contained_points[:, 2] < 0.50))
__bp_indicies = np.argwhere(__boundary_points[..., 2] > 0.48)

plt.close(fig=6)
plt.figure(num=6).add_subplot(projection='3d')
plt.gca().scatter(__contained_points[__cp_indicies][0:-1:1, ..., 0], __contained_points[__cp_indicies][0:-1:1, ..., 1], __contained_points[__cp_indicies][0:-1:1, ..., 2], marker='1')
plt.gca().scatter(__boundary_points[__bp_indicies][..., 0], __boundary_points[__bp_indicies][..., 1], __boundary_points[__bp_indicies][..., 2], marker='.')
plt.show()

In [ ]:
__cp_indicies = np.argwhere(np.logical_and(__contained_points[:, 2] > 0.58, __contained_points[:, 2] < 0.60))
__bp_indicies = np.argwhere(__boundary_points[..., 2] > 0.58)

plt.close(fig=7)
plt.figure(num=7).add_subplot(projection='3d')
plt.gca().scatter(__contained_points[__cp_indicies][0:-1:1, ..., 0], __contained_points[__cp_indicies][0:-1:1, ..., 1], __contained_points[__cp_indicies][0:-1:1, ..., 2], marker='1')
plt.gca().scatter(__boundary_points[__bp_indicies][..., 0], __boundary_points[__bp_indicies][..., 1], __boundary_points[__bp_indicies][..., 2], marker='.')
plt.show()

In [ ]:
__r_corners = np.reshape(cube_rotation.apply(np.reshape(__corners, newshape=(-1, 3)), inverse=True), newshape=__corners.shape)
__r_contained_points = cube_rotation.apply(__contained_points, inverse=True)


plt.close(fig=7)
plt.figure(num=7).add_subplot(projection='3d')
plt.gca().voxels(__r_corners[..., 0], __r_corners[..., 1], __r_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__r_contained_points[0:-1:3,..., 0], __r_contained_points[0:-1:3,..., 1], __r_contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
__sr_corners = __r_corners * np.sqrt(3) * edge_len
__sr_contained_points = __r_contained_points * np.sqrt(3) * edge_len

plt.close(fig=8)
plt.figure(num=8).add_subplot(projection='3d')
plt.gca().voxels(__sr_corners[..., 0], __sr_corners[..., 1], __sr_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__sr_contained_points[0:-1:3,..., 0], __sr_contained_points[0:-1:3,..., 1], __sr_contained_points[0:-1:3,..., 2], marker='o')
plt.show()

In [ ]:
__sr_outband_points_collection = []
for point in __sr_contained_points:
    if(np.any(point < -0)):
        __sr_outband_points_collection.append(point)
__sr_outband_points = np.array(__sr_outband_points_collection)

plt.close(fig=9)
plt.figure(num=9).add_subplot(projection='3d')
plt.gca().voxels(__sr_corners[..., 0], __sr_corners[..., 1], __sr_corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
#plt.gca().scatter(__boundary_points[..., 0], __boundary_points[..., 1], __boundary_points[..., 2], marker='o')
plt.gca().scatter(__sr_outband_points[..., 0], __sr_outband_points[..., 1], __sr_outband_points[..., 2], marker='o')
plt.show()

In [ ]:
__obp_indicies = np.argwhere(__sr_contained_points < 0)
__outband_points = __contained_points[np.argwhere(__sr_contained_points < 0)]
print(__obp_indicies.shape)
print(__outband_points.shape)

plt.close(fig=10)
plt.figure(num=10).add_subplot(projection='3d')
plt.gca().voxels(__corners[..., 0], __corners[..., 1], __corners[..., 2], cube[:1, :1, :1], facecolors=colors[-1:, :1, :1], edgecolors=colors[:1, :1, -1:], shade=False, alpha=0.3)
plt.gca().scatter(__outband_points[..., 0], __outband_points[..., 1], __outband_points[..., 2], marker='o')
plt.show()

In [ ]:
__line_r_range = np.linspace(-3, 3, num=100)
__line_angle = np.pi / 6
plt.close(fig=11)
plt.figure(num=11)
plt.gca().axis('equal')
plt.gca().grid(True)
plt.gca().plot(__line_r_range * np.cos(__line_angle), __line_r_range * np.sin(__line_angle), label='forward')
plt.gca().plot(__line_r_range * np.cos(__line_angle - np.pi/2), __line_r_range * np.sin(__line_angle - np.pi/2), label='rotated')
plt.gca().plot(__line_r_range * np.cos(__line_angle), -__line_r_range * np.sin(__line_angle), label='inverse')
plt.gcf().legend()
plt.show()

In [ ]:
temp = np.array(border_crossings)
__d_temp = np.array(crossings_dist)
__rc_temp = np.array(planed_vectors)
#print(np.argwhere(__fill_voxels == True))
for i in range(1600, 1600):
    print(f"{__rc_temp[i]}: {__d_temp[i]}")
#temp = np.reshape(temp, newshape=(3, -1))

_border_crossings = np.reshape(np.array(border_crossings), (-1, 3))

surf_base = np.meshgrid(np.linspace(-5, 5, 10), np.linspace(-5, 5, 10))
surf_points = np.zeros(shape=(6, 10, 10, 3))
for i in range(planes.shape[0]):
    surf_points[i, :, :, 0] = surf_base[0]
    surf_points[i, :, :, 1] = surf_base[1]
    surf_points[i, :, :, 2] = (surf_base[0] * planes[i][0] + surf_base[1] * planes[i][1] - planes[i][3]) / (-planes[i][2])

_surf_points = np.reshape(surf_points, (6, -1, 3))

#for j in range(planes.shape[0]):
#    for i in range(planes.shape[0]):
#        print(np.dot(planes[j][0:3], planes[i][0:3]))

plt.close(fig=8)
plt.figure(num=8).add_subplot(projection='3d')\
    .voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), cylinder_grid[..., 2], cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1], shade=False)

#plt.gca().scatter(temp[0:100, 0], temp[0:100, 1], temp[0:100, 2], marker='^')
surf_cmaps = ["Greens", "Reds", "Blues", "ocean", "Oranges", "Greys"]

for i in []:
    plt.gca().plot_surface(surf_points[i][:, :, 0], surf_points[i][:, :, 1], surf_points[i][:, :, 2], cmap=surf_cmaps[i])
#for p in range(surf_points.shape[0]):
#    plt.gca().plot_surface(surf_points[p][:, :, 0], surf_points[p][:, :, 1], surf_points[p][:, :, 2], cmap=surf_cmaps[p])
plt.show()

In [ ]:
plt.close(fig=9)
plt.figure(num=9).add_subplot(projection='3d')
plt.gca().voxels(cylinder_grid[..., 0] * np.cos(cylinder_grid[..., 1]), cylinder_grid[..., 0] * np.sin(cylinder_grid[..., 1]), cylinder_grid[..., 2], cylinder[:-1, :-1, :-1], facecolors=cylinder_colors[:-1, :-1, :-1], shade=False, alpha=0.4)
plt.show()